## **apple/FastVLM-1.5B**

apple/FastVLM-1.5B is an advanced vision-language model introduced at CVPR 2025, featuring the innovative FastViTHD hybrid vision encoder for quick, high-resolution image processing with fewer tokens, which enables efficient multimodal interaction. The FastVLM-1.5B variant significantly enhances performance over the baseline and smaller models, delivering excellent results across several benchmarks—including Ai2D, ScienceQA, MMMU, VQAv2, ChartQA, TextVQA, InfoVQA, DocVQA, OCRBench, RealWorldQA, and SeedBench-Img—while maintaining a streamlined inference pipeline and supporting easy integration with Hugging Face Transformers for PyTorch-based deployments. FastVLM outpaces competing models like LLaVA-OneVision-0.5B and Cambrian-1-8B in speed and efficiency, making it a top choice for research and real-world vision-language applications.

`accelerator: 1 X NVIDIA T4`

| Image1 | Image2 |
|--------|--------|
| ![Screenshot 2025-08-30 at 12-50-11 Gradio.png](https://cdn-uploads.huggingface.co/production/uploads/65bb837dbfb878f46c77de4c/HpSwXWjiO4FpkBaR-KRCs.png) | ![Screenshot 2025-08-30 at 12-49-41 (anonymous) - output_426f8ad8-53ee-4609-9d55-6629ac37b055.pdf.png](https://cdn-uploads.huggingface.co/production/uploads/65bb837dbfb878f46c77de4c/ckjXfTLlPtea0gwt8HGIQ.png) |

*notebook by: [prithivMLmods](https://huggingface.co/prithivMLmods)*

### **Install packages**

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git \
             git+https://github.com/huggingface/accelerate.git \
             git+https://github.com/huggingface/peft.git \
             transformers-stream-generator huggingface_hub albumentations \
             pyvips-binary qwen-vl-utils sentencepiece opencv-python docling-core \
             python-docx torchvision safetensors matplotlib num2words \

!pip install xformers requests pymupdf hf_xet spaces pyvips pillow gradio \
             einops torch fpdf timm av decord bitsandbytes reportlab
#Hold tight, this will take around 2-3 minutes.

### **Run Demo App**

In [ ]:
import spaces
import json
import math
import os
import traceback
from io import BytesIO
from typing import Any, Dict, List, Optional, Tuple
import re
import time
from threading import Thread
from io import BytesIO
import uuid
import tempfile

import gradio as gr
import requests
import torch
from PIL import Image
import fitz
import numpy as np

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Image as RLImage, Paragraph, Spacer
from reportlab.lib.units import inch

# --- Constants and Model Setup ---
MAX_INPUT_TOKEN_LENGTH = 4096
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA_VISIBLE_DEVICES=", os.environ.get("CUDA_VISIBLE_DEVICES"))
print("torch.__version__ =", torch.__version__)
print("torch.version.cuda =", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("cuda device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("current device:", torch.cuda.current_device())
    print("device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

print("Using device:", device)


# --- Model Loading ---
# We are only loading the apple/FastVLM-1.5B model
MODEL_ID_FV = "apple/FastVLM-1.5B"
IMAGE_TOKEN_INDEX_FV = -200 # Special token index for the image placeholder

print(f"Loading model: {MODEL_ID_FV}")
tokenizer_fv = AutoTokenizer.from_pretrained(MODEL_ID_FV, trust_remote_code=True)
model_fv = AutoModelForCausalLM.from_pretrained(
    MODEL_ID_FV,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
).eval()
print("Model loaded successfully.")


# --- PDF Generation and Preview Utility Function ---
def generate_and_preview_pdf(image: Image.Image, text_content: str, font_size: int, line_spacing: float, alignment: str, image_size: str):
    """
    Generates a PDF, saves it, and then creates image previews of its pages.
    Returns the path to the PDF and a list of paths to the preview images.
    """
    if image is None or not text_content or not text_content.strip():
        raise gr.Error("Cannot generate PDF. Image or text content is missing.")

    # --- 1. Generate the PDF ---
    temp_dir = tempfile.gettempdir()
    pdf_filename = os.path.join(temp_dir, f"output_{uuid.uuid4()}.pdf")
    doc = SimpleDocTemplate(
        pdf_filename,
        pagesize=A4,
        rightMargin=inch, leftMargin=inch,
        topMargin=inch, bottomMargin=inch
    )
    styles = getSampleStyleSheet()
    style_normal = styles["Normal"]
    style_normal.fontSize = int(font_size)
    style_normal.leading = int(font_size) * line_spacing
    style_normal.alignment = {"Left": 0, "Center": 1, "Right": 2, "Justified": 4}[alignment]

    story = []

    img_buffer = BytesIO()
    image.save(img_buffer, format='PNG')
    img_buffer.seek(0)

    page_width, _ = A4
    available_width = page_width - 2 * inch
    image_widths = {
        "Small": available_width * 0.3,
        "Medium": available_width * 0.6,
        "Large": available_width * 0.9,
    }
    img_width = image_widths[image_size]
    # Create a ReportLab Image object, handling potential transparency
    img = RLImage(img_buffer, width=img_width, height=image.height * (img_width / image.width))
    story.append(img)
    story.append(Spacer(1, 12))

    # Clean the text for PDF generation
    cleaned_text = re.sub(r'#+\s*', '', text_content).replace("*", "")
    text_paragraphs = cleaned_text.split('\n')

    for para in text_paragraphs:
        if para.strip():
            story.append(Paragraph(para, style_normal))

    doc.build(story)

    # --- 2. Render PDF pages as images for preview ---
    preview_images = []
    try:
        pdf_doc = fitz.open(pdf_filename)
        for page_num in range(len(pdf_doc)):
            page = pdf_doc.load_page(page_num)
            pix = page.get_pixmap(dpi=150)
            preview_img_path = os.path.join(temp_dir, f"preview_{uuid.uuid4()}_p{page_num}.png")
            pix.save(preview_img_path)
            preview_images.append(preview_img_path)
        pdf_doc.close()
    except Exception as e:
        print(f"Error generating PDF preview: {e}")

    return pdf_filename, preview_images


# --- Core Application Logic ---
@spaces.GPU
def process_document(
    image: Image.Image,
    prompt_input: str,
    max_new_tokens: int,
    temperature: float,
    top_p: float,
    top_k: int,
    repetition_penalty: float
):
    """
    Main function that handles model inference for FastVLM.
    """
    if image is None:
        return "Please upload an image.", ""
    if not prompt_input or not prompt_input.strip():
        return "Please enter a prompt.", ""

    # --- Prepare inputs for Apple FastVLM ---

    # 1. Build the chat prompt string, including the <image> placeholder
    messages = [{"role": "user", "content": f"<image>\n{prompt_input}"}]
    rendered = tokenizer_fv.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=False
    )

    # 2. Split the text around the placeholder
    pre, post = rendered.split("<image>", 1)

    # 3. Tokenize the text parts separately
    pre_ids  = tokenizer_fv(pre,  return_tensors="pt", add_special_tokens=False).input_ids
    post_ids = tokenizer_fv(post, return_tensors="pt", add_special_tokens=False).input_ids

    # 4. Create the special image token and splice the parts together
    img_tok = torch.tensor([[IMAGE_TOKEN_INDEX_FV]], dtype=pre_ids.dtype)
    input_ids = torch.cat([pre_ids, img_tok, post_ids], dim=1).to(model_fv.device)
    attention_mask = torch.ones_like(input_ids, device=model_fv.device)

    # 5. Process the image using the model's vision tower
    pixel_values = model_fv.get_vision_tower().image_processor(images=image.convert("RGB"), return_tensors="pt")["pixel_values"]
    pixel_values = pixel_values.to(model_fv.device, dtype=model_fv.dtype)

    # --- Generate the response ---
    with torch.no_grad():
        out = model_fv.generate(
            inputs=input_ids,
            attention_mask=attention_mask,
            images=pixel_values,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            do_sample=True if temperature > 0 else False,
        )

    response = tokenizer_fv.decode(out[0], skip_special_tokens=True)

    # The response from FastVLM often includes the prompt; this cleans it up.
    cleaned_response = response.split("assistant")[-1].strip()

    return cleaned_response, cleaned_response


# --- Gradio UI Definition ---
def create_gradio_interface():
    """Builds and returns the Gradio web interface."""
    css = """
    .main-container { max-width: 1400px; margin: 0 auto; }
    .process-button { border: none !important; color: white !important; font-weight: bold !important; background-color: blue !important;}
    .process-button:hover { background-color: darkblue !important; transform: translateY(-2px) !important; box-shadow: 0 4px 8px rgba(0,0,0,0.2) !important; }
    #gallery { min-height: 400px; }
    """
    with gr.Blocks(theme="bethecloud/storj_theme", css=css) as demo:
        gr.HTML(f"""
        <div class="title" style="text-align: center">
            <h1>apple/FastVLM-1.5B🍏</h1>
            <p style="font-size: 1.1em; color: #6b7280; margin-bottom: 0.6em;">
                Image Content Analysis and Scene Understanding </b>
            </p>
        </div>
        """)

        with gr.Row():
            # Left Column (Inputs)
            with gr.Column(scale=1):
                prompt_input = gr.Textbox(label="Query Input", placeholder="✦︎ Enter the prompt.", value="Describe this image in detail.")
                image_input = gr.Image(label="Upload Image", type="pil", sources=['upload'])

                with gr.Accordion("Advanced Settings", open=False):
                    max_new_tokens = gr.Slider(minimum=64, maximum=2048, value=512, step=32, label="Max New Tokens")
                    temperature = gr.Slider(label="Temperature", minimum=0.1, maximum=2.0, step=0.1, value=0.7)
                    top_p = gr.Slider(label="Top-p (nucleus sampling)", minimum=0.05, maximum=1.0, step=0.05, value=0.9)
                    top_k = gr.Slider(label="Top-k", minimum=1, maximum=100, step=1, value=50)
                    repetition_penalty = gr.Slider(label="Repetition penalty", minimum=1.0, maximum=2.0, step=0.05, value=1.1)

                with gr.Accordion("PDF Export Settings", open=False):
                    font_size = gr.Dropdown(choices=["8", "10", "12", "14", "16", "18"], value="12", label="Font Size")
                    line_spacing = gr.Dropdown(choices=[1.0, 1.15, 1.5, 2.0], value=1.15, label="Line Spacing")
                    alignment = gr.Dropdown(choices=["Left", "Center", "Right", "Justified"], value="Justified", label="Text Alignment")
                    image_size = gr.Dropdown(choices=["Small", "Medium", "Large"], value="Medium", label="Image Size in PDF")

                process_btn = gr.Button("🚀 Process Image", variant="primary", elem_classes=["process-button"], size="lg")
                clear_btn = gr.Button("🗑️ Clear All", variant="secondary")

            # Right Column (Outputs)
            with gr.Column(scale=2):
                with gr.Tabs() as tabs:
                    with gr.Tab("📝 Extracted Content"):
                        raw_output = gr.Textbox(label="Model Output", interactive=False, lines=15, show_copy_button=True)

                        gr.Markdown("[prithivMLmods🤗](https://huggingface.co/prithivMLmods)")

                    with gr.Tab("📰 Markdown Preview"):
                        with gr.Accordion("(Result.md)", open=True):
                            markdown_output = gr.Markdown()

                    with gr.Tab("📋 PDF Preview"):
                        generate_pdf_btn = gr.Button("📄 Generate PDF & Render", variant="primary")
                        pdf_output_file = gr.File(label="Download Generated PDF", interactive=False)
                        pdf_preview_gallery = gr.Gallery(label="PDF Page Preview", show_label=True, elem_id="gallery", columns=2, object_fit="contain", height="auto")

        # Event Handlers
        def clear_all_outputs():
            return None, "", "Model output will appear here.", "", None, None

        process_btn.click(
            fn=process_document,
            inputs=[image_input, prompt_input, max_new_tokens, temperature, top_p, top_k, repetition_penalty],
            outputs=[raw_output, markdown_output]
        )

        generate_pdf_btn.click(
            fn=generate_and_preview_pdf,
            inputs=[image_input, raw_output, font_size, line_spacing, alignment, image_size],
            outputs=[pdf_output_file, pdf_preview_gallery]
        )

        clear_btn.click(
            clear_all_outputs,
            outputs=[image_input, prompt_input, raw_output, markdown_output, pdf_output_file, pdf_preview_gallery]
        )
    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    # Use queue() for better handling of multiple users
    demo.queue(max_size=20).launch(share=True, show_error=True)